In [ ]:
# @title 0. Configuración Inicial y Librerías

# --- 0.1 Instalar Librerías ---
# Instalamos/actualizamos las librerías necesarias.
# - google-generativeai: Para interactuar con la API de Gemini.
# - pandas: Para manejar el archivo CSV.
# - numpy: Para operaciones numéricas (arrays de embeddings).
# - scikit-learn: Incluye PCA y otras utilidades (aunque usaremos umap por separado).
# - plotly: Para gráficos interactivos.
# - umap-learn: Para la reducción de dimensionalidad UMAP (buena para visualización).
# - nltk: Para dividir frases en sentencias (tokenización).
print("--- 0.1 Instalando librerías necesarias ---")
!pip install -q -U google-generativeai pandas numpy scikit-learn plotly umap-learn nltk
print("Librerías base instaladas/actualizadas.\n")

# --- 0.2 Importar Librerías ---
print("--- 0.2 Importando librerías ---")
try:
    import google.generativeai as genai
    import pandas as pd
    import numpy as np
    import os # Para manejo de rutas de archivo
    import json # Para trabajar con la columna JSON del CSV
    import random # Para seleccionar una fila de muestra
    import plotly.express as px # Para gráficos fáciles e interactivos
    from sklearn.decomposition import PCA # Como opción de reducción de dimensionalidad
    import umap # Para reducción de dimensionalidad UMAP
    import nltk # Para procesamiento de lenguaje natural (tokenización de frases)
    from google.colab import widgets # Para el dropdown de selección de frases
    import textwrap # Para acortar texto largo en las impresiones
    from google.colab import userdata # Para gestionar la API Key de forma segura

    print("Librerías principales importadas correctamente.\n")
except ImportError as e:
    print(f"(!) Error importando una librería esencial: {e}")
    print("Por favor, verifica la instalación en el paso 0.1.")
    # Podrías detener la ejecución aquí si una librería crítica falla
    raise # Detiene la ejecución de la celda si falla una importación crítica

# --- 0.3 Configurar API Key de Gemini ---
print("--- 0.3 Configurando API Key de Gemini ---")
api_key_configured = False # Bandera para saber si se configuró
try:
    # Intenta obtener la API Key desde los Secrets de Colab (método recomendado)
    # Debes haberla guardado con el nombre 'GOOGLE_API_KEY'
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key de Gemini configurada correctamente desde Secrets.")
    api_key_configured = True
except userdata.SecretNotFoundError:
    print("(!) Advertencia: Secret 'GOOGLE_API_KEY' no encontrado.")
    print("Por favor, ve a los 'Secrets' de Colab (Panel izquierdo -> Icono de llave) y añade tu API key.")
    print("Las llamadas a la API fallarán sin una clave válida.")
except Exception as e:
    print(f"(!) Ocurrió un error inesperado al configurar la API Key: {e}")
    print("Las llamadas a la API podrían fallar.")

# --- 0.4 Descargar Recursos NLTK ---
# Necesitamos el tokenizador 'punkt' para dividir texto en frases/sentencias.
print("\n--- 0.4 Descargando recursos de NLTK (punkt) ---")
try:
    nltk.download('punkt', quiet=True) # quiet=True evita mucho texto en la salida
    # Verificar que se pueda usar (opcional pero bueno)
    nltk.sent_tokenize("Esto es una prueba. Funciona.")
    print("Recursos NLTK 'punkt' descargados y verificados.")
except Exception as e:
    print(f"(!) Error al descargar/verificar recursos de NLTK 'punkt': {e}")
    print("La funcionalidad para dividir frases en sentencias podría fallar más adelante.")

# --- 0.5 Definir Constantes de Modelos (Ajusta si prefieres otros) ---
# Usamos nombres de modelos recomendados y generalmente disponibles.
# Puedes cambiarlos si tienes acceso a versiones específicas o experimentales.
EMBEDDING_MODEL_NAME = 'embedding-001' # O el modelo experimental cuando deja hacerlo.
LLM_MODEL_NAME = 'gemini-1.5-flash'   # Modelo rápido y capaz para generar el Meta-CoT.
                                      # Alternativa: 'gemini-1.5-pro-latest' (más potente, potencialmente más lento/costoso)
print(f"\n--- 0.5 Constantes Definidas ---")
print(f"Modelo de Embedding a usar: '{EMBEDDING_MODEL_NAME}'")
print(f"Modelo LLM para Meta-CoT: '{LLM_MODEL_NAME}'")

# --- 0.6 Verificación Final ---
print("\n--- 0.6 Verificación Final ---")
if not api_key_configured:
    print("(!) ATENCIÓN: La API Key no está configurada. Debes añadirla a los Secrets para continuar.")
else:
    print("Configuración inicial completada. ¡Listo para el siguiente paso!")

In [ ]:
# @title 1. Carga, Limpieza y Exploración Inicial del CSV desde `sample_data`

import pandas as pd
import os
import random
import textwrap

# --- Parámetros del Usuario ---
# @markdown 1. **Verifica la subida:** Asegúrate de que tu archivo CSV (con análisis CoT/JSON) esté en la carpeta `sample_data` en el panel izquierdo.
# @markdown 2. **Introduce el NOMBRE EXACTO** de tu archivo CSV (incluyendo `.csv`):
nombre_archivo_csv_analisis = "datos_filtrados.csv" # @param {type:"string"}

# @markdown 3. **Columnas Esperadas:** Confirma los nombres de las columnas clave (sensible a mayúsculas/minúsculas):
columna_frase_original = "frase" # @param {type:"string"}
columna_cot = "Complex.CoT" # @param {type:"string"}
columna_respuesta_json = "respuesta" # @param {type:"string"}
# --- Fin Parámetros ---

# Variable global para almacenar el DataFrame cargado y limpio
df_analisis = None

# Construir la ruta completa al archivo dentro del entorno de Colab
ruta_csv_completa = os.path.join("/content/sample_data", nombre_archivo_csv_analisis)

print(f"--- 1.1 Intentando cargar datos desde: {ruta_csv_completa} ---")

# Verificar si el archivo existe
if not nombre_archivo_csv_analisis or not nombre_archivo_csv_analisis.endswith('.csv'):
     print("(!) Error: Debes proporcionar un nombre de archivo CSV válido (que termine en .csv).")
elif not columna_frase_original or not columna_cot or not columna_respuesta_json:
     print("(!) Error: Debes especificar los nombres de las tres columnas clave (frase, CoT, respuesta JSON).")
elif os.path.exists(ruta_csv_completa):
    try:
        # Leer el CSV
        df_cargado_temp = pd.read_csv(ruta_csv_completa)
        print(f"-> Archivo CSV cargado inicialmente con {df_cargado_temp.shape[0]} filas.")

        # Verificar la existencia de las columnas clave
        columnas_necesarias = [columna_frase_original, columna_cot, columna_respuesta_json]
        columnas_faltantes = [col for col in columnas_necesarias if col not in df_cargado_temp.columns]

        if not columnas_faltantes:
            print(f"-> Columnas esperadas ({', '.join(columnas_necesarias)}) encontradas.")

            # --- 1.1.1 Limpieza de Duplicados Exactos ---
            print("\n--- 1.1.1 Verificando y eliminando duplicados exactos ---")
            filas_antes = len(df_cargado_temp)
            # drop_duplicates sin 'subset' considera TODAS las columnas
            df_limpio = df_cargado_temp.drop_duplicates(keep='first') # Mantener la primera ocurrencia
            filas_despues = len(df_limpio)
            filas_eliminadas = filas_antes - filas_despues

            if filas_eliminadas > 0:
                 print(f"-> Se eliminaron {filas_eliminadas} fila(s) que eran duplicados exactos en todas las columnas.")
                 print(f"-> El DataFrame ahora tiene {filas_despues} filas.")
            else:
                 print("-> No se encontraron filas completamente duplicadas.")

            # Asignar el DataFrame limpio a la variable global
            df_analisis = df_limpio
            print("-> DataFrame limpio asignado a 'df_analisis'.")

        else: # Si faltaban columnas necesarias
            print(f"(!) Error CRÍTICO: Faltan las siguientes columnas esenciales en el CSV: {', '.join(columnas_faltantes)}")
            print(f"   Columnas encontradas en el archivo: {list(df_cargado_temp.columns)}")
            print("   No se continuará con la limpieza ni asignación.")

    except pd.errors.EmptyDataError:
        print(f"(!) Error: El archivo CSV en '{ruta_csv_completa}' parece estar vacío.")
    except Exception as e:
        print(f"(!) Error inesperado al leer o procesar el archivo CSV: {e}")

else: # Si el archivo no existe
    print(f"(!) Error CRÍTICO: Archivo '{nombre_archivo_csv_analisis}' NO encontrado en '/content/sample_data/'.")
    # ... (mensajes de ayuda para subir archivo) ...

# --- 1.2 Exploración (Si el DataFrame se limpió y asignó correctamente) ---
if df_analisis is not None:
    print("\n--- 1.2 Información General del DataFrame Limpio ('df_analisis') ---") # Cambiado a "Limpio"
    print(f"-> Dimensiones: {df_analisis.shape[0]} filas x {df_analisis.shape[1]} columnas")

    # Contar frases únicas (ahora debería coincidir con el número de filas si las duplicadas eran exactas)
    if columna_frase_original in df_analisis.columns:
        num_frases_unicas = df_analisis[columna_frase_original].nunique()
        print(f"-> Número de frases únicas en la columna '{columna_frase_original}': {num_frases_unicas}")
        if num_frases_unicas != df_analisis.shape[0]:
             print(f"   (!) Nota: Todavía hay {df_analisis.shape[0] - num_frases_unicas} frases que aparecen en múltiples filas, pero estas filas difieren en las columnas CoT o respuesta.")
    else:
         print(f"(!) Advertencia: La columna '{columna_frase_original}' no se encontró para contar únicos.")

    # --- 1.3 Muestra Aleatoria (del DataFrame limpio) ---
    print("\n--- 1.3 Muestra Aleatoria de una Fila del DataFrame Limpio ---")
    if not df_analisis.empty:
        try:
            random_row = df_analisis.sample(1).iloc[0]
            print(f"Mostrando contenido de la Fila con índice original: {random_row.name}") # El índice puede haber cambiado

            for col_name in columnas_necesarias:
                cell_content = str(random_row[col_name])
                shortened_content = textwrap.shorten(cell_content, width=200, placeholder="...")
                print(f"\n  [{col_name}]:")
                print(f"    {shortened_content}")

        except Exception as e:
            print(f"(!) Error al generar la muestra aleatoria: {e}")
    else:
        print("-> El DataFrame está vacío después de la limpieza, no se puede mostrar muestra.")

    print("\n--- Fin Exploración Inicial ---")

else:
    print("\n(!) El DataFrame 'df_analisis' no pudo ser cargado o limpiado. Revisa los errores anteriores.")

In [99]:
# @title 2. Selección Interactiva de la Frase

# Importar ipywidgets y display
import ipywidgets as widgets # <--- CORRECCIÓN: Usar ipywidgets
from IPython.display import display, clear_output
import pandas as pd # Asegurarse de que pandas esté disponible por si acaso

# Variable global para almacenar el widget dropdown
dropdown_selector_frase = None

# Recuperar el nombre de la columna de la celda anterior (o definirlo si es necesario)
# Asumiendo que 'columna_frase_original' se definió en la celda anterior (Chunk 1)
# Si no, descomenta y ajusta la línea siguiente:
# columna_frase_original = "frase"

print("--- 2.1 Preparando Selector de Frases ---")

# Verificar que df_analisis exista y contenga la columna de frases
if 'df_analisis' in globals() and isinstance(df_analisis, pd.DataFrame) and not df_analisis.empty:
    if columna_frase_original in df_analisis.columns:

        # Obtener la lista de frases únicas
        # Convertir a string para evitar problemas con tipos mixtos si los hubiera
        lista_frases_unicas = df_analisis[columna_frase_original].astype(str).unique().tolist()

        # Ordenar alfabéticamente para facilitar la búsqueda
        lista_frases_unicas.sort()

        if lista_frases_unicas:
            print(f"-> Encontradas {len(lista_frases_unicas)} frases únicas para seleccionar.")

            # Crear el widget Dropdown usando ipywidgets
            dropdown_selector_frase = widgets.Dropdown(
                options=lista_frases_unicas,
                description='Elige una frase:',
                # Ajustar el estilo para manejar frases potencialmente largas
                style={'description_width': 'initial'},
                layout={'width': '95%'} # Usar un porcentaje para mejor adaptabilidad
                # disabled=False
            )

            print("-> Widget Dropdown creado.")

            # --- Mostrar el Widget ---
            print("\n--- 2.2 Por favor, selecciona una frase del menú desplegable ---")
            display(dropdown_selector_frase)
            print("--- Ejecuta la siguiente celda (Chunk 3) DESPUÉS de hacer tu selección ---")

        else:
            print(f"(!) Error: La columna '{columna_frase_original}' existe, pero no contiene frases únicas válidas después de la extracción.")

    else:
        print(f"(!) Error: La columna '{columna_frase_original}' definida en el Chunk 1 no se encuentra en 'df_analisis'.")
        print("   Asegúrate de que el nombre de la columna sea correcto y el Chunk 1 se haya ejecutado sin errores.")
else:
    print("(!) Error: El DataFrame 'df_analisis' no existe o está vacío.")
    print("   Asegúrate de que el Chunk 1 se haya ejecutado correctamente y haya cargado los datos.")


# --- Nota Opcional: Cómo acceder al valor seleccionado ---
# En la siguiente celda (Chunk 3), accederemos al valor así:
# frase_seleccionada = dropdown_selector_frase.value
# print(f"Valor seleccionado actualmente: {dropdown_selector_frase.value}") # Para probar ahora mismo si quieres

--- 2.1 Preparando Selector de Frases ---
-> Encontradas 122 frases únicas para seleccionar.
-> Widget Dropdown creado.

--- 2.2 Por favor, selecciona una frase del menú desplegable ---


Dropdown(description='Elige una frase:', layout=Layout(width='95%'), options=('Abrazar mi inmensa soledad en e…

--- Ejecuta la siguiente celda (Chunk 3) DESPUÉS de hacer tu selección ---


In [100]:
# @title 3 (Modificado): Recuperar Datos, Segmentar Texto y Generar Meta-CoT

import google.generativeai as genai
import json
import pandas as pd
import textwrap # Para mostrar texto de forma más limpia
import re # Para procesar la respuesta del LLM

# --- 3.1 Recuperar Selección y Datos ---
print("--- 3.1 Recuperando datos de la frase/texto seleccionado ---") # Cambiado 'frase' a 'frase/texto'

# (El código para verificar variables y obtener frase_seleccionada, cot_original, json_respuesta_str es el mismo)
# ... (código de verificación inicial y obtención de datos de la fila) ...

# Verificar si las variables necesarias existen
if 'dropdown_selector_frase' not in globals() or dropdown_selector_frase is None:
    print("(!) Error: El widget Dropdown ('dropdown_selector_frase') no existe.")
    # ... (resto del manejo de error) ...
elif 'df_analisis' not in globals() or df_analisis is None or df_analisis.empty:
    print("(!) Error: El DataFrame 'df_analisis' no existe o está vacío.")
    # ... (resto del manejo de error) ...
else:
    frase_seleccionada = dropdown_selector_frase.value
    print(f"-> Texto seleccionado: '{textwrap.shorten(frase_seleccionada, width=100)}'")
    try:
        fila_datos = df_analisis.loc[df_analisis[columna_frase_original].astype(str) == frase_seleccionada].iloc[0]
        print("-> Fila de datos encontrada.")
        cot_original = fila_datos[columna_cot]
        json_respuesta_str = fila_datos[columna_respuesta_json]

        # --- 3.2 Parsear la Respuesta JSON Original ---
        print("\n--- 3.2 Intentando parsear la respuesta JSON original ---")
        # (El código para parsear parsed_json y manejar json_parse_error es el mismo)
        # ... (código de parseo JSON) ...
        parsed_json = None # Inicializar
        json_parse_error = None
        if pd.isna(json_respuesta_str):
             print("-> La columna 'respuesta' contiene NaN (Nulo).")
        elif isinstance(json_respuesta_str, str):
            try:
                parsed_json = json.loads(json_respuesta_str)
                if isinstance(parsed_json, (dict, list)):
                     print("-> JSON parseado correctamente.")
                else:
                    print(f"(!) Advertencia: JSON parseado no es dict/list (Tipo: {type(parsed_json)}).")
                    parsed_json = None
            except json.JSONDecodeError as e:
                json_parse_error = str(e)
                print(f"(!) Advertencia: La columna '{columna_respuesta_json}' no contiene JSON válido: {json_parse_error}")
            except Exception as e:
                 json_parse_error = f"Error inesperado al parsear JSON: {e}"
                 print(f"(!) Error inesperado al parsear '{columna_respuesta_json}': {e}")
                 parsed_json = None
        else:
            print(f"(!) Advertencia: Contenido de '{columna_respuesta_json}' no es string ni NaN.")


        # --- 3.3 Generar Segmentación y Meta-CoT ---
        print("\n--- 3.3 Preparando y generando Segmentación y Meta-Análisis CoT ---")

        # Variables para almacenar los resultados procesados
        lista_unidades_detectadas = []
        meta_cot_resumen = "(No generado)" # Default

        if not api_key_configured:
             print("(!) Error CRÍTICO: La API Key de Gemini no está configurada.")
        else:
             # --- Nuevo Prompt con Tarea de Segmentación ---
             prompt_segmentacion_y_meta_cot = f"""
Contexto: Estoy creando una visualización semántica interactiva para el siguiente texto (que puede ser una frase, párrafo o estrofa). Necesito identificar sus frases/unidades principales y obtener un resumen de su análisis lingüístico previo.

Texto Original Completo:
'''
{frase_seleccionada}
'''

Análisis Lingüístico Previo (CoT Original extraído de un análisis anterior):
--- CoT Original ---
{cot_original if pd.notna(cot_original) else 'No disponible'}
--- Fin CoT Original ---

Datos Estructurados Adicionales (extraídos de un JSON del análisis anterior, si estaban disponibles y eran válidos):
--- JSON Parseado ---
{json.dumps(parsed_json, indent=2, ensure_ascii=False) if parsed_json else 'JSON no disponible, inválido, o contenía un error.'}
{f' (Error al parsear JSON original: {json_parse_error})' if json_parse_error else ''}
--- Fin JSON Parseado ---

Tarea Principal (Realiza en este orden):
1.  **Segmentación en Unidades:** Basándote en el Texto Original y el CoT, identifica y lista las frases gramaticales completas o unidades semánticas principales dentro del texto. Si el texto es claramente una sola unidad/frase, indícalo explícitamente como "1. Unidad Única: [texto completo]". Si hay múltiples unidades, numéralas (1. [Texto Unidad 1], 2. [Texto Unidad 2], ...). Sé preciso al extraer el texto de cada unidad.
2.  **Meta-Análisis CoT Conciso:** Después de la lista de segmentación, añade una línea separadora "--- Meta-Análisis ---" y luego genera un resumen (máximo 3-4 puntos clave) enfocado en aspectos semánticos/contextuales relevantes para la visualización (Naturaleza principal, Ambigüedad, Complejidad/Características, Conclusión Semántica). Basa este resumen SÓLO en el CoT Original y el JSON proporcionado.
3.  **Palabras unidas en sentido figurado:** Después del Meta-Análisis, y considerando también el CoT original, revisa si el texto tiene predominantemente un caracter literario, poético, musical, etc. Si es así, entonces menciona casos significativos de asociación no literal, más figurada, de palabras y frases si aplica.
**Instrucciones de Formato:**
*   Primero, la lista numerada de unidades segmentadas (o la indicación de Unidad Única).
*   Luego, la línea separadora "--- Meta-Análisis ---".
*   Finalmente, el Meta-Análisis CoT en puntos numerados o con viñetas, incluyendo las palabras y frases unidas en sentido figurado si es que aplica.

**Ejemplo de Salida Esperada (si hay múltiples unidades):**
1. Texto de la primera frase o unidad detectada.
2. Texto de la segunda frase o unidad detectada.
--- Meta-Análisis ---
*   Naturaleza: [Resumen]
*   Ambigüedad: [Resumen]
*   Complejidad: [Resumen]
*   Revisión crítica: [Resumen]

**Ejemplo de Salida Esperada (si es una sola unidad):**
1. Unidad Única: [Texto completo original]
--- Meta-Análisis ---
*   Naturaleza: [Resumen]
*   Ambigüedad: [Resumen]
*   Complejidad: [Resumen]
*   Revisión Crítica: [Resumen]

**INICIA LA SALIDA:**
"""

             try:
                 print(f"-> Inicializando modelo LLM: '{LLM_MODEL_NAME}'...")
                 model_llm = genai.GenerativeModel(LLM_MODEL_NAME)
                 generation_config = genai.types.GenerationConfig(temperature=0.2)

                 print("-> Generando Segmentación y Meta-CoT (esto puede tardar)...")
                 response = model_llm.generate_content(
                     prompt_segmentacion_y_meta_cot,
                     generation_config=generation_config
                 )
                 respuesta_completa_llm = response.text

                 # --- 3.4 Procesar la Respuesta del LLM ---
                 print("\n--- 3.4 Procesando la respuesta del LLM ---")
                 segmentacion_str = ""
                 meta_cot_resumen_temp = "" # Usar temporal para evitar sobreescribir default

                 # Buscar el separador
                 separador = "--- Meta-Análisis ---"
                 partes = respuesta_completa_llm.split(separador, 1) # Dividir solo en el primer separador

                 if len(partes) == 2:
                     segmentacion_str = partes[0].strip()
                     meta_cot_resumen_temp = partes[1].strip()
                     print("-> Separador encontrado. Extrayendo Segmentación y Meta-Análisis.")
                 else:
                     # Si no encontró el separador, asumir que toda la respuesta es el Meta-CoT
                     # y que no pudo segmentar o lo indicó dentro del texto.
                     print("(!) Advertencia: No se encontró el separador '--- Meta-Análisis ---' en la respuesta.")
                     print("   Asumiendo que la respuesta completa es el Meta-Análisis o incluye la segmentación.")
                     segmentacion_str = respuesta_completa_llm # Guardar todo por si acaso
                     meta_cot_resumen_temp = respuesta_completa_llm # O asignar solo una parte si se prefiere

                 # Extraer las unidades de la parte de segmentación
                 unidades_extraidas_temp = []
                 # Usar regex para encontrar líneas numeradas al inicio
                 # Ajusta la regex si el formato de salida del LLM varía
                 matches = re.findall(r"^\s*\d+\.\s*(.*)", segmentacion_str, re.MULTILINE)
                 if matches:
                     for match in matches:
                         # Comprobar si es la indicación de unidad única
                         if match.lower().startswith("unidad única:"):
                              texto_unidad = match[len("unidad única:"):].strip()
                              # Podríamos decidir añadir el texto completo o la frase seleccionada
                              unidades_extraidas_temp.append(frase_seleccionada) # Añadir el original completo
                              print(f"   - Detectada indicación de Unidad Única.")
                              break # Salir si es unidad única
                         else:
                              unidades_extraidas_temp.append(match.strip())
                     print(f"   - Extraídas {len(unidades_extraidas_temp)} unidades/frases de la segmentación.")
                 else:
                      print("   (!) No se encontraron líneas numeradas claras en la sección de segmentación.")
                      # Si no hay líneas numeradas, podría ser una sola unidad no marcada explícitamente
                      # o el LLM no siguió el formato. Como fallback, usar la frase original.
                      if len(unidades_extraidas_temp) == 0:
                           print("      -> Asumiendo una única unidad (el texto completo).")
                           unidades_extraidas_temp.append(frase_seleccionada)

                 # Asignar a las variables finales
                 lista_unidades_detectadas = unidades_extraidas_temp
                 meta_cot_resumen = meta_cot_resumen_temp if meta_cot_resumen_temp else "(Meta-Análisis no extraído)"


                 # --- 3.5 Mostrar Resultados ---
                 print("\n" + "="*70)
                 print("Resultados del Chunk 3:")
                 print("="*70)
                 print("Segmentación Detectada:")
                 print("-"*70)
                 if lista_unidades_detectadas:
                     for i, unidad in enumerate(lista_unidades_detectadas):
                         print(f"{i+1}. {textwrap.fill(unidad, width=80, subsequent_indent='   ')}")
                 else:
                     print("(No se detectaron unidades separadas)")

                 print("\n" + "-"*70)
                 print("Meta-Análisis CoT (Resumen):")
                 print("-"*70)
                 # Usar display(Markdown(...)) para el resumen formateado
                 from IPython.display import display, Markdown
                 display(Markdown(meta_cot_resumen))
                 print("="*70)


             except Exception as e:
                 print(f"(!) Error durante la generación o procesamiento con LLM '{LLM_MODEL_NAME}': {e}")
                 # Guardar error en las variables para saber que falló
                 lista_unidades_detectadas = []
                 meta_cot_resumen = f"Error al generar/procesar: {e}"

    # ... (Manejo de errores para IndexError y otros al buscar la fila) ...
    except IndexError:
        print(f"(!) Error: No se encontró fila para: '{frase_seleccionada}'")
    except Exception as e:
        print(f"(!) Error inesperado al procesar fila: {e}")

# --- Fin Chunk 3 (Modificado) ---
if 'lista_unidades_detectadas' not in globals(): lista_unidades_detectadas = [] # Asegurar que exista
if 'meta_cot_resumen' not in globals(): meta_cot_resumen = "(No disponible)" # Asegurar que exista

print(f"\n--- Fin del Proceso del Chunk 3 Modificado (Detectadas {len(lista_unidades_detectadas)} unidades) ---")

--- 3.1 Recuperando datos de la frase/texto seleccionado ---
-> Texto seleccionado: 'Para realizar la configuración óptima del dispositivo, asegúrese de verificar que los puertos [...]'
-> Fila de datos encontrada.

--- 3.2 Intentando parsear la respuesta JSON original ---
-> JSON parseado correctamente.

--- 3.3 Preparando y generando Segmentación y Meta-Análisis CoT ---
-> Inicializando modelo LLM: 'gemini-1.5-flash'...
-> Generando Segmentación y Meta-CoT (esto puede tardar)...

--- 3.4 Procesando la respuesta del LLM ---
-> Separador encontrado. Extrayendo Segmentación y Meta-Análisis.
   - Extraídas 2 unidades/frases de la segmentación.

Resultados del Chunk 3:
Segmentación Detectada:
----------------------------------------------------------------------
1. Para realizar la configuración óptima del dispositivo, asegúrese de verificar
   que los puertos de entrada estén correctamente alineados con los conectores
   del cable HDMI.
2. los cuales suelen ubicarse en la parte posterior

* Naturaleza: Instrucción técnica formal para la configuración de un dispositivo electrónico.  El texto pertenece a un manual de usuario o guía de configuración, con el objetivo de guiar al usuario en la conexión correcta del dispositivo.
* Ambigüedad: Ausencia de ambigüedad significativa. El significado es claro y conciso, dirigido a evitar problemas de conexión.
* Complejidad:  Estructura gramatical compleja, con una oración principal y una subordinada (que a su vez contiene una oración relativa).  Sin embargo, la complejidad gramatical no afecta la claridad del mensaje.  El texto utiliza vocabulario técnico específico del ámbito de la electrónica.
* Conclusión Semántica: El texto cumple eficazmente su función instructiva, proporcionando instrucciones precisas y concisas para evitar fallos en la conexión del dispositivo.


* Palabras unidas en sentido figurado: No hay palabras o frases unidas en sentido figurado. El lenguaje es completamente literal y técnico.  No presenta características literarias, poéticas o musicales.


--- Fin del Proceso del Chunk 3 Modificado (Detectadas 2 unidades) ---


In [101]:
# @title 4 (Revisado v3): Preparación y Generación de Embeddings (Palabras con Detalles JSON y Unidades)

import google.generativeai as genai
import numpy as np
import pandas as pd
import json
import time
import textwrap

# --- Reutilizar Función de Embedding ---
# (Asumiendo que la función get_embeddings_batch_local está definida arriba o en celda previa)
# ... (código de la función get_embeddings_batch_local) ...
def get_embeddings_batch_local(texts, model_name, task_type="RETRIEVAL_DOCUMENT", max_retries=2, initial_delay=2):
    # ... (código completo de la función) ...
    embeddings_list = [None] * len(texts); # ... (resto del código de la función) ...
    if not texts: print("-> No hay textos."); return embeddings_list # ...
    if 'api_key_configured' not in globals() or not api_key_configured: print("(!) Error API Key."); return embeddings_list # ...
    print(f"-> Solicitando embeddings para {len(texts)} textos ({model_name}, {task_type})..."); # ...
    current_texts = list(texts); indices = list(range(len(texts))); batch_size = 100; all_results = {}; # ...
    for i in range(0, len(current_texts), batch_size): # ...
        batch_texts = current_texts[i:i + batch_size]; batch_indices = indices[i:i + batch_size]; # ...
        print(f"   Procesando batch {i//batch_size + 1} (tamaño: {len(batch_texts)})..."); # ...
        for attempt in range(max_retries + 1): # ...
            try: # ...
                result = genai.embed_content(model=f"models/{model_name}", content=batch_texts, task_type=task_type); # ...
                if 'embedding' in result and len(result['embedding']) == len(batch_texts): # ...
                    print(f"      -> Embeddings OK (intento {attempt + 1})."); # ...
                    for j, emb in enumerate(result['embedding']): all_results[batch_indices[j]] = list(emb); # ...
                    break; # ...
            except Exception as e: # ...
                print(f"   (!) Error API (Batch {i//batch_size + 1}, Intento {attempt + 1}): {e}"); # ...
                error_str = str(e).lower(); # ...
                if "model not found" in error_str or "api key not valid" in error_str or "permission denied" in error_str or "invalid argument" in error_str: print("      -> Error definitivo."); break; # ...
                if attempt < max_retries: delay = initial_delay * (2 ** attempt); print(f"      -> Reintentando en {delay} seg..."); time.sleep(delay); # ...
                else: print(f"      -> Máximo reintentos."); break; # ...
    for original_idx in range(len(texts)): # ...
        if original_idx in all_results: embeddings_list[original_idx] = all_results[original_idx]; # ...
    num_exitosos = sum(1 for emb in embeddings_list if emb is not None); # ...
    print(f"-> Proceso finalizado. {num_exitosos}/{len(texts)} embeddings generados."); # ...
    if num_exitosos < len(texts): print(f"   (!) {len(texts) - num_exitosos} texto(s) fallaron."); # ...
    return embeddings_list; # ...

# --- 4.1 Recuperar Datos Necesarios ---
print("--- 4.1 Recuperando datos (JSON, Unidades, Texto Completo) ---")
# ... (código de verificación de variables sin cambios) ...
variables_ok = True # ... (resto del código de verificación) ...
if 'frase_seleccionada' not in globals(): print("(!) Error: Falta 'frase_seleccionada'."); variables_ok = False # ...
if 'parsed_json' not in globals(): print("(!) Error: Falta 'parsed_json'.") # ... (puede ser None) ...
if 'lista_unidades_detectadas' not in globals(): print("(!) Error: Falta 'lista_unidades_detectadas'."); variables_ok = False # ...
if 'columna_respuesta_json' not in globals(): print("(!) Error: Falta 'columna_respuesta_json'."); variables_ok = False # ...

palabras_info_detallada = [] # <--- CAMBIO: Guardará diccionarios con TODOS los detalles
textos_unidades = []
texto_completo = None

if variables_ok:
    texto_completo = frase_seleccionada
    textos_unidades = lista_unidades_detectadas if isinstance(lista_unidades_detectadas, list) else []

    # --- 4.2 Extraer Información DETALLADA de Palabras del JSON --- ## MODIFICADO ##
    print("\n--- 4.2 Extrayendo información DETALLADA de palabras del JSON ---")
    campos_posibles = set() # Para rastrear todos los campos encontrados

    if isinstance(parsed_json, list):
        print(f"-> JSON es lista con {len(parsed_json)} elementos.")
        for index, item in enumerate(parsed_json):
            if isinstance(item, dict):
                palabra_data = {'indice': index} # Guardar índice original del JSON
                # Extraer todos los campos presentes en el item del JSON
                for key, value in item.items():
                    # Guardamos el valor si no es None y (si es string/lista) no está vacío
                    # Excepciones: palabra_analizada y categoria se guardan aunque estén vacíos (para asegurar existencia)
                    if key in ['palabra_analizada', 'categoria'] or \
                       (value is not None and not isinstance(value, (str, list)) or value): # Guarda si no es None Y (no es str/list O no está vacío)
                         palabra_data[key] = value
                         campos_posibles.add(key)

                # Asegurar campos mínimos y renombrar 'palabra' para consistencia
                if 'palabra_analizada' not in palabra_data:
                     palabra_data['texto'] = item.get('texto', f'Palabra_{index}_?') # Usar 'texto' como nombre estándar
                     if palabra_data['texto'] == f'Palabra_{index}_?': print(f"   (!) Advertencia: JSON[{index}] sin 'palabra_analizada'/'text'.")
                else:
                     palabra_data['texto'] = palabra_data.pop('palabra_analizada') # Renombrar a 'texto'

                if 'categoria' not in palabra_data:
                     palabra_data['categoria'] = 'desconocida'

                palabras_info_detallada.append(palabra_data) # <--- Guardar en la nueva lista
            else:
                 print(f"   (!) Advertencia: JSON[{index}] no es diccionario.")
        print(f"-> Extraídos datos detallados para {len(palabras_info_detallada)} palabras.")
        print(f"-> Campos JSON encontrados: {sorted(list(campos_posibles))}")
        if palabras_info_detallada:
            # Mostrar llaves del primer diccionario como ejemplo
            print(f"   Ejemplo campos Palabra 0: {list(palabras_info_detallada[0].keys())}")
    elif parsed_json is None: print("-> JSON no disponible/inválido.")
    else: print(f"(!) Error: JSON parseado no es lista (tipo: {type(parsed_json)}).")

else:
     print("(!) No se pueden continuar los preparativos.")

# --- 4.3 Preparar Lista Unificada de Textos para Embedding ---
print("\n--- 4.3 Preparando lista unificada de textos ---")
lista_unificada_textos = []
lista_tipos_texto = []
lista_indices_ref = []
lista_categorias_temp = [] # Lista temporal solo para palabras

if variables_ok:
    # 1. Añadir Palabras (solo el texto)
    if palabras_info_detallada: # <--- Usar la nueva lista
        print(f"-> Añadiendo texto de {len(palabras_info_detallada)} palabras...")
        for info in palabras_info_detallada:
            lista_unificada_textos.append(info.get('texto','?')) # Añadir el texto de la palabra
            lista_tipos_texto.append("Palabra")
            lista_indices_ref.append(info.get('indice', -1)) # Índice original del JSON
            lista_categorias_temp.append(info.get('categoria', 'desconocida')) # Guardar categoría temporalmente
    else:
         print("-> No se añadieron palabras.")

    # 2. Añadir Unidades Detectadas
    # ... (código para añadir unidades a lista_unificada_textos y lista_tipos_texto sin cambios) ...
    num_unidades_anadidas = 0
    if len(textos_unidades) > 1:
        print(f"-> Añadiendo {len(textos_unidades)} unidades detectadas...")
        for idx, unidad_texto in enumerate(textos_unidades):
             if unidad_texto != texto_completo:
                lista_unificada_textos.append(unidad_texto); lista_tipos_texto.append("Unidad"); lista_indices_ref.append(idx + 1); lista_categorias_temp.append(None); num_unidades_anadidas += 1
             else: print(f"   (Omitiendo unidad {idx+1})")
        print(f"   -> {num_unidades_anadidas} unidades añadidas.")
    # ... (manejo caso 1 unidad o 0 unidades) ...
    elif len(textos_unidades) == 1: print("-> Solo 1 unidad detectada, se usará TextoCompleto.")
    else: print("-> No se detectaron unidades separadas.")

    # 3. Añadir Texto Completo Original
    # ... (código para añadir texto_completo sin cambios) ...
    if texto_completo:
        print("-> Añadiendo texto completo..."); lista_unificada_textos.append(texto_completo); lista_tipos_texto.append("TextoCompleto"); lista_indices_ref.append(0); lista_categorias_temp.append(None)
    else: print("(!) No se pudo añadir texto completo.")

    print(f"-> Total textos unificados para embeddear: {len(lista_unificada_textos)}")

else:
     print("(!) No se preparó la lista unificada.")


# --- 4.4 Generar Embeddings para la Lista Unificada ---
print("\n--- 4.4 Generando embeddings para lista unificada ---")
# ... (código para determinar task_type_auto sin cambios) ...
num_palabras = sum(1 for t in lista_tipos_texto if t == 'Palabra'); num_otros = len(lista_unificada_textos) - num_palabras; task_type_auto = "SEMANTIC_SIMILARITY" if num_palabras > num_otros else "RETRIEVAL_DOCUMENT"; print(f"   (Usando task_type='{task_type_auto}')")

lista_embeddings_unificada = get_embeddings_batch_local(
    lista_unificada_textos, model_name=EMBEDDING_MODEL_NAME, task_type=task_type_auto
)

# --- 4.5 Estructurar Resultados Combinados (con Detalles JSON) --- ## MODIFICADO ##
print("\n--- 4.5 Estructurando resultados combinados (con detalles JSON) ---")
resultados_combinados = []
num_embeddings_fallidos_total = 0

if len(lista_unificada_textos) == len(lista_embeddings_unificada):
    palabra_info_idx = 0 # Contador para acceder a palabras_info_detallada
    for i in range(len(lista_unificada_textos)):
        embedding = lista_embeddings_unificada[i]
        if embedding is None:
             num_embeddings_fallidos_total += 1
             print(f"  (!) Falló embedding para: ({lista_tipos_texto[i]} {lista_indices_ref[i]}) '{textwrap.shorten(lista_unificada_textos[i], 60)}'")
             continue # Saltar si no hay embedding

        # Crear registro base
        record = {
            "texto": lista_unificada_textos[i],
            "tipo": lista_tipos_texto[i],
            "indice_ref": lista_indices_ref[i],
            "embedding_vector": np.array(embedding)
        }

        # Si es una palabra, añadir TODOS los detalles de palabras_info_detallada
        if record['tipo'] == 'Palabra':
            if palabra_info_idx < len(palabras_info_detallada):
                 # Fusionar el record base con el diccionario de detalles de la palabra
                 # Dando prioridad a los valores ya en record (texto, tipo, indice_ref, embedding)
                 # y añadiendo los demás del diccionario de detalles
                 detalles_palabra = palabras_info_detallada[palabra_info_idx]
                 # Crear un nuevo diccionario para evitar modificar el original
                 record_completo = record.copy()
                 # Añadir detalles, evitando sobreescribir claves base si ya existen en detalles_palabra
                 for key, value in detalles_palabra.items():
                      if key not in record_completo: # Añadir solo si no es una clave base ya presente
                           record_completo[key] = value
                 # Asegurar que 'categoria' esté presente, usando la de detalles si existe
                 record_completo['categoria'] = detalles_palabra.get('categoria', 'desconocida')
                 record = record_completo # Usar el registro completo
                 palabra_info_idx += 1
            else:
                 print(f"(!) Advertencia: Discrepancia mapeo embedding palabra {i}.")
                 record['categoria'] = 'error_mapeo'

        # Si es Unidad o TextoCompleto, asegurar que 'categoria' sea None
        elif record['tipo'] in ['Unidad', 'TextoCompleto']:
             record['categoria'] = None

        resultados_combinados.append(record)

    if resultados_combinados:
        # Crear DataFrame a partir de la lista de diccionarios
        df_embeddings_combinado = pd.DataFrame(resultados_combinados)

        # Rellenar posibles NaNs en columnas JSON (opcional, pero puede ayudar)
        # Identificar columnas que no sean las base o el vector
        columnas_base = ['texto', 'tipo', 'indice_ref', 'embedding_vector', 'categoria', 'indice']
        columnas_json = [col for col in df_embeddings_combinado.columns if col not in columnas_base]
        # df_embeddings_combinado[columnas_json] = df_embeddings_combinado[columnas_json].fillna("N/A") # Llenar con string "N/A"

        print(f"\n-> Creado DataFrame 'df_embeddings_combinado' con {len(df_embeddings_combinado)} elementos válidos.")
        print(f"   Columnas: {list(df_embeddings_combinado.columns)}")
    # ... (resto del código de manejo de errores y df vacío) ...

else:
    # ... (código de manejo de discrepancia o fallo total) ...
    print(f"(!) Discrepancia textos/embeddings o fallo total.")
    df_embeddings_combinado = pd.DataFrame()

# --- Fin Chunk 4 (Revisado v3) ---
print(f"\n--- Fin del Proceso del Chunk 4 Revisado v3 ({num_embeddings_fallidos_total} fallos) ---")

--- 4.1 Recuperando datos (JSON, Unidades, Texto Completo) ---

--- 4.2 Extrayendo información DETALLADA de palabras del JSON ---
-> JSON es lista con 54 elementos.
-> Extraídos datos detallados para 54 palabras.
-> Campos JSON encontrados: ['aumentativo_comun', 'caso', 'categoria', 'definicion_contextual', 'definicion_funcion', 'definicion_general', 'diminutivo_comun', 'ejemplos_uso', 'funcion', 'genero', 'gerundio', 'grado', 'infinitivo', 'lemma', 'modifica_a', 'modo', 'numero', 'palabra_analizada', 'participio', 'persona', 'referente_aproximado', 'subtipo', 'tiempo', 'tipo', 'tonicidad', 'transitividad', 'usos_comunes']
   Ejemplo campos Palabra 0: ['indice', 'categoria', 'tipo', 'usos_comunes', 'ejemplos_uso', 'texto']

--- 4.3 Preparando lista unificada de textos ---
-> Añadiendo texto de 54 palabras...
-> Añadiendo 2 unidades detectadas...
   -> 2 unidades añadidas.
-> Añadiendo texto completo...
-> Total textos unificados para embeddear: 57

--- 4.4 Generando embeddings para lis

In [102]:
# @title 5 (Revisado v2): Reducción de Dimensionalidad para Embeddings Combinados

import numpy as np
import pandas as pd
import umap
from sklearn.decomposition import PCA

# --- 5.1 Preparar Datos Combinados para Reducción ---
print("--- 5.1 Preparando embeddings COMBINADOS (Palabras, Unidades, Texto Completo) ---")

embeddings_2d_umap_combinado = None
embeddings_2d_pca_combinado = None
reduction_combinada_successful = False # Bandera para este chunk

# Verificar si tenemos el DataFrame combinado del Chunk 4 (Revisado v2)
if 'df_embeddings_combinado' in globals() and isinstance(df_embeddings_combinado, pd.DataFrame) and not df_embeddings_combinado.empty:
    if 'embedding_vector' in df_embeddings_combinado.columns:
        # Extraer solo los vectores de embedding válidos (no None)
        valid_embeddings_series = df_embeddings_combinado['embedding_vector'].dropna()

        if not valid_embeddings_series.empty:
            embedding_vectors_list_combinado = valid_embeddings_series.tolist()

            # Apilar los vectores en un único array NumPy 2D
            try:
                # Asegurarse de que todos los elementos son arrays NumPy antes de apilar
                valid_vectors_combinado = [vec for vec in embedding_vectors_list_combinado if isinstance(vec, np.ndarray)]
                if len(valid_vectors_combinado) != len(embedding_vectors_list_combinado):
                     print(f"(!) Advertencia: Se descartaron {len(embedding_vectors_list_combinado) - len(valid_vectors_combinado)} elementos no válidos antes de apilar.")

                if not valid_vectors_combinado:
                     print("(!) Error: No hay vectores de embedding válidos para apilar.")
                     reduction_combinada_possible = False
                else:
                    embeddings_array_combinado = np.stack(valid_vectors_combinado, axis=0)
                    print(f"-> Array de embeddings combinados preparado. Forma: {embeddings_array_combinado.shape}")

                    # Verificar si tenemos al menos 2 puntos para reducir
                    if embeddings_array_combinado.shape[0] >= 2:
                        reduction_combinada_possible = True
                        print(f"-> Se puede realizar reducción para {embeddings_array_combinado.shape[0]} elementos combinados.")
                    else:
                        print("-> Solo hay 1 elemento con embedding válido. No se necesita reducción.")
                        reduction_combinada_possible = False
                        # Asignar coordenadas (0,0) para el único punto
                        embeddings_2d_umap_combinado = np.array([[0.0, 0.0]])
                        embeddings_2d_pca_combinado = np.array([[0.0, 0.0]])
                        reduction_combinada_successful = True # Consideramos éxito tener las coords

            except ValueError as e:
                 print(f"(!) Error al apilar los vectores de embedding combinados: {e}")
                 reduction_combinada_possible = False
            except Exception as e:
                 print(f"(!) Error inesperado al preparar el array combinado: {e}")
                 reduction_combinada_possible = False

        else: # Si después de dropna() no queda nada
             print("(!) Error: No hay embeddings válidos en 'df_embeddings_combinado' después de eliminar Nulos.")
             reduction_combinada_possible = False
    else:
        print("(!) Error: La columna 'embedding_vector' no se encontró en 'df_embeddings_combinado'.")
        reduction_combinada_possible = False
else:
    print("(!) Error: El DataFrame 'df_embeddings_combinado' no existe o está vacío.")
    print("   Asegúrate de que el Chunk 4 (Revisado v2) se haya ejecutado correctamente.")
    reduction_combinada_possible = False

# --- 5.2 Aplicar Reducción de Dimensionalidad (si es posible y necesario) ---
if reduction_combinada_possible:
    print("\n--- 5.2 Aplicando Reducción de Dimensionalidad a embeddings combinados ---")

    # --- UMAP ---
    print("   -> Calculando reducción combinada con UMAP...")
    try:
        n_samples = embeddings_array_combinado.shape[0]
        n_neighbors_umap = min(15, max(2, n_samples - 1)) if n_samples > 2 else 1
        print(f"      (Usando n_neighbors={n_neighbors_umap})")

        umap_reducer_combinado = umap.UMAP(
            n_components=2, n_neighbors=n_neighbors_umap, min_dist=0.1,
            metric='cosine', random_state=42, n_jobs=1
        )
        embeddings_2d_umap_combinado = umap_reducer_combinado.fit_transform(embeddings_array_combinado)
        print(f"      -> Reducción UMAP combinada completada. Forma: {embeddings_2d_umap_combinado.shape}")
        reduction_combinada_successful = True
    except Exception as e:
        print(f"   (!) Error durante la reducción UMAP combinada: {e}")
        embeddings_2d_umap_combinado = None
        if not reduction_combinada_successful: print("      Intentando PCA como alternativa...")


    # --- PCA ---
    print("\n   -> Calculando reducción combinada con PCA...")
    try:
        pca_reducer_combinado = PCA(n_components=2, random_state=42)
        embeddings_2d_pca_combinado = pca_reducer_combinado.fit_transform(embeddings_array_combinado)
        print(f"      -> Reducción PCA combinada completada. Forma: {embeddings_2d_pca_combinado.shape}")
        if not reduction_combinada_successful: reduction_combinada_successful = True
    except Exception as e:
        print(f"   (!) Error durante la reducción PCA combinada: {e}")
        embeddings_2d_pca_combinado = None


# --- 5.3 Añadir Coordenadas al DataFrame Combinado ---
# Asegurarse de que el DataFrame original exista para añadirle las columnas
if 'df_embeddings_combinado' in globals() and isinstance(df_embeddings_combinado, pd.DataFrame) and not df_embeddings_combinado.empty:
    if reduction_combinada_successful:
        print("\n--- 5.3 Añadiendo coordenadas 2D al DataFrame 'df_embeddings_combinado' ---")

        # Crear DataFrames temporales con las coordenadas y el índice original
        # Esto es más seguro si hubo NaNs y se filtraron al crear embeddings_array_combinado
        indices_validos = valid_embeddings_series.index # Índices de las filas con embeddings válidos

        try:
            # Añadir UMAP si se calcularon
            if embeddings_2d_umap_combinado is not None and embeddings_2d_umap_combinado.shape[0] == len(indices_validos):
                 df_coords_umap = pd.DataFrame(embeddings_2d_umap_combinado, columns=['umap_x', 'umap_y'], index=indices_validos)
                 df_embeddings_combinado = df_embeddings_combinado.join(df_coords_umap)
                 print("   -> Coordenadas UMAP (umap_x, umap_y) añadidas/actualizadas.")
            else:
                 print("   (!) No se añadieron coordenadas UMAP (falló cálculo o forma/índices no coinciden).")
                 if 'umap_x' not in df_embeddings_combinado.columns: df_embeddings_combinado['umap_x'] = np.nan
                 if 'umap_y' not in df_embeddings_combinado.columns: df_embeddings_combinado['umap_y'] = np.nan

            # Añadir PCA si se calcularon
            if embeddings_2d_pca_combinado is not None and embeddings_2d_pca_combinado.shape[0] == len(indices_validos):
                 df_coords_pca = pd.DataFrame(embeddings_2d_pca_combinado, columns=['pca_x', 'pca_y'], index=indices_validos)
                 df_embeddings_combinado = df_embeddings_combinado.join(df_coords_pca)
                 print("   -> Coordenadas PCA (pca_x, pca_y) añadidas/actualizadas.")
            else:
                 print("   (!) No se añadieron coordenadas PCA (falló cálculo o forma/índices no coinciden).")
                 if 'pca_x' not in df_embeddings_combinado.columns: df_embeddings_combinado['pca_x'] = np.nan
                 if 'pca_y' not in df_embeddings_combinado.columns: df_embeddings_combinado['pca_y'] = np.nan

            # Rellenar posibles NaNs introducidos por el join si alguna coordenada falló
            # df_embeddings_combinado[['umap_x', 'umap_y', 'pca_x', 'pca_y']] = df_embeddings_combinado[['umap_x', 'umap_y', 'pca_x', 'pca_y']].fillna(0.0) # Opcional: rellenar con 0

            # Mostrar ejemplo
            # print("\n   DataFrame 'df_embeddings_combinado' con coordenadas:")
            # print(df_embeddings_combinado[['texto', 'tipo', 'umap_x', 'umap_y', 'pca_x', 'pca_y']].head())

        except Exception as e:
              print(f"(!) Error al añadir coordenadas al DataFrame combinado: {e}")

    else: # Si la reducción no fue exitosa pero el DataFrame existe
         print("\n--- 5.3 No se realizó o falló la reducción de dimensionalidad combinada ---")
         if 'umap_x' not in df_embeddings_combinado.columns: df_embeddings_combinado['umap_x'] = np.nan
         if 'umap_y' not in df_embeddings_combinado.columns: df_embeddings_combinado['umap_y'] = np.nan
         if 'pca_x' not in df_embeddings_combinado.columns: df_embeddings_combinado['pca_x'] = np.nan
         if 'pca_y' not in df_embeddings_combinado.columns: df_embeddings_combinado['pca_y'] = np.nan
         print("   -> Columnas de coordenadas añadidas como NaN.")

# --- Fin Chunk 5 (Revisado v2) ---
print("\n--- Fin del Proceso del Chunk 5 Revisado v2 ---")

--- 5.1 Preparando embeddings COMBINADOS (Palabras, Unidades, Texto Completo) ---
-> Array de embeddings combinados preparado. Forma: (57, 768)
-> Se puede realizar reducción para 57 elementos combinados.

--- 5.2 Aplicando Reducción de Dimensionalidad a embeddings combinados ---
   -> Calculando reducción combinada con UMAP...
      (Usando n_neighbors=15)
      -> Reducción UMAP combinada completada. Forma: (57, 2)

   -> Calculando reducción combinada con PCA...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



      -> Reducción PCA combinada completada. Forma: (57, 2)

--- 5.3 Añadiendo coordenadas 2D al DataFrame 'df_embeddings_combinado' ---
   -> Coordenadas UMAP (umap_x, umap_y) añadidas/actualizadas.
   -> Coordenadas PCA (pca_x, pca_y) añadidas/actualizadas.

--- Fin del Proceso del Chunk 5 Revisado v2 ---


In [104]:
# @title 6 (Revisado v3): Creación del DataFrame Final con Tooltip Dinámico

import pandas as pd
import textwrap
import numpy as np # Para isnan si fuera necesario

# --- Función para formatear Tooltip ---
def format_tooltip_details(row):
    """Formatea el tooltip HTML basado en la fila de datos."""
    if row.get('tipo') != 'Palabra':
        # Tooltip simple para Unidad o TextoCompleto
        # Mantener este simple por ahora, podríamos añadir más si es necesario
        return f"<b>Tipo:</b> {row.get('tipo', 'N/A')}<br><b>Texto:</b> {textwrap.shorten(str(row.get('texto','?')), width=60, placeholder='...')}"

    # --- Construir tooltip detallado para Palabras ---
    details = []
    # NO añadimos Categoría aquí, lo haremos en el hovertemplate

    # --- Campos Comunes ---
    # Usar <b> para las etiquetas
    if pd.notna(row.get('lemma')): details.append(f"<b>Lema:</b> {row['lemma']}")
    if pd.notna(row.get('genero')): details.append(f"<b>Género:</b> {row['genero']}")
    if pd.notna(row.get('numero')): details.append(f"<b>Número:</b> {row['numero']}")
    if pd.notna(row.get('tipo')) and row.get('tipo') != row.get('categoria'):
         details.append(f"<b>Tipo Espec.:</b> {row['tipo']}")

    # --- Campos de Verbos ---
    if pd.notna(row.get('infinitivo')): details.append(f"<b>Infinitivo:</b> {row['infinitivo']}")
    if pd.notna(row.get('modo')): details.append(f"<b>Modo:</b> {row['modo']}")
    if pd.notna(row.get('tiempo')): details.append(f"<b>Tiempo:</b> {row['tiempo']}")
    if pd.notna(row.get('persona')): details.append(f"<b>Persona:</b> {row['persona']}")
    if pd.notna(row.get('participio')): details.append(f"<b>Participio:</b> {row['participio']}")
    if pd.notna(row.get('gerundio')): details.append(f"<b>Gerundio:</b> {row['gerundio']}")
    if pd.notna(row.get('transitividad')): details.append(f"<b>Transitiv.:</b> {row['transitividad']}")

    # --- Campos de Adjetivos / Adverbios ---
    if pd.notna(row.get('grado')): details.append(f"<b>Grado:</b> {row['grado']}")
    if pd.notna(row.get('apocope')): details.append(f"<b>Apócope:</b> {row['apocope']}")

    # --- Campos de Pronombres ---
    if pd.notna(row.get('caso')): details.append(f"<b>Caso:</b> {row['caso']}")
    if pd.notna(row.get('tonicidad')): details.append(f"<b>Tonicidad:</b> {row['tonicidad']}")
    if pd.notna(row.get('referente_aproximado')): details.append(f"<b>Referente:</b> {row['referente_aproximado']}")

    # --- Campos de Determinantes ---
    if pd.notna(row.get('subtipo')) and row.get('categoria') == 'determinante':
        details.append(f"<b>Subtipo:</b> {row['subtipo']}")
    if pd.notna(row.get('distancia')): details.append(f"<b>Distancia:</b> {row['distancia']}")

    # --- Campos de Sustantivos ---
    if pd.notna(row.get('diminutivo_comun')): details.append(f"<b>Diminutivo:</b> {row['diminutivo_comun']}")
    if pd.notna(row.get('aumentativo_comun')): details.append(f"<b>Aumentativo:</b> {row['aumentativo_comun']}")

    # --- Otros Campos ---
    if pd.notna(row.get('subtipo')) and row.get('categoria') in ['conjunción', 'interjección']:
         details.append(f"<b>Subtipo:</b> {row['subtipo']}")
    if pd.notna(row.get('emocion_tipica')): details.append(f"<b>Emoción:</b> {row['emocion_tipica']}")
    if pd.notna(row.get('funcion')): details.append(f"<b>Función:</b> {row['funcion']}")
    if pd.notna(row.get('descripcion')): details.append(f"<b>Desc.:</b> {row['descripcion']}")

    # --- Definiciones (Opcional, pueden ser largas) ---
    if pd.notna(row.get('definicion_contextual')): details.append(f"<b>Def. Context.:</b> {textwrap.shorten(row['definicion_contextual'], width=80, placeholder='...')}")
    if pd.notna(row.get('definicion_general')): details.append(f"<b>Def. Gral.:</b> {textwrap.shorten(row['definicion_general'], width=80, placeholder='...')}")

    # Unir los detalles con saltos de línea HTML
    tooltip_html = "<br>".join(details)

    return tooltip_html if details else "<i>(Sin detalles adicionales)</i>"

# --- Inicio del Chunk 6 ---
print("--- 6.1 Creando DataFrame Final 'df_plot_final' (con Tooltip Dinámico) ---")

df_plot_final = None

# Verificar DataFrame combinado del paso anterior
if 'df_embeddings_combinado' in globals() and isinstance(df_embeddings_combinado, pd.DataFrame) and not df_embeddings_combinado.empty:

    # Verificar si las columnas de coordenadas esenciales existen
    has_umap_coords = 'umap_x' in df_embeddings_combinado.columns and 'umap_y' in df_embeddings_combinado.columns and not df_embeddings_combinado['umap_x'].isnull().all()
    has_pca_coords = 'pca_x' in df_embeddings_combinado.columns and 'pca_y' in df_embeddings_combinado.columns and not df_embeddings_combinado['pca_x'].isnull().all()

    if has_umap_coords or has_pca_coords:
        df_plot_final = df_embeddings_combinado.copy()
        print(f"-> Copiando df_embeddings_combinado ({len(df_plot_final)} filas) a df_plot_final.")

        # --- 6.2 Crear Columna de Tooltip Formateado ---
        print("   -> Creando columna 'tooltip_details' formateada dinámicamente...")
        try:
            df_plot_final['tooltip_details'] = df_plot_final.apply(format_tooltip_details, axis=1)
            print("      -> Columna 'tooltip_details' creada exitosamente.")
        except Exception as e:
            print(f"   (!) Error al crear 'tooltip_details': {e}")
            df_plot_final['tooltip_details'] = "Error al formatear" # Fallback

        # --- 6.3 Asegurar otras Columnas Necesarias ---
        columnas_base_plot = ['texto', 'tipo', 'indice_ref', 'categoria', 'tooltip_details'] # Columnas descriptivas + tooltip
        columnas_coords_plot = []
        if has_umap_coords: columnas_coords_plot.extend(['umap_x', 'umap_y', 'umap_x_3d', 'umap_y_3d', 'umap_z_3d']) # Incluir 3D si existen
        if has_pca_coords: columnas_coords_plot.extend(['pca_x', 'pca_y', 'pca_x_3d', 'pca_y_3d', 'pca_z_3d']) # Incluir 3D si existen

        columnas_plot_final = columnas_base_plot + columnas_coords_plot

        columnas_faltantes_plot = [col for col in columnas_plot_final if col not in df_plot_final.columns]
        if not columnas_faltantes_plot:
             print("   -> Todas las columnas necesarias para visualización están presentes o se manejarán.")
             if 'frase_seleccionada' in globals(): df_plot_final['frase_madre'] = frase_seleccionada
             else: df_plot_final['frase_madre'] = 'Desconocida'
             columnas_plot_final.append('frase_madre') # Añadir a la lista final

             print("\n-> DataFrame 'df_plot_final' listo.")
             # Mostrar cabecera con columnas clave, incluyendo el nuevo tooltip
             print("\n   Primeras filas de 'df_plot_final' (con tooltip_details):")
             cols_to_show = ['texto', 'tipo', 'categoria', 'tooltip_details'] + [c for c in columnas_coords_plot if c in df_plot_final.columns]
             try:
                 # Acortar tooltip para la vista previa del head
                 df_head_preview = df_plot_final[cols_to_show].head().copy()
                 if 'tooltip_details' in df_head_preview.columns:
                      df_head_preview['tooltip_details'] = df_head_preview['tooltip_details'].apply(lambda x: textwrap.shorten(str(x).replace('<br>', ' | '), width=60, placeholder='...'))
                 print(df_head_preview.to_markdown(index=False))
             except ImportError: print(df_plot_final[cols_to_show].head())
             except Exception as e: print(f"(!) Error al mostrar head: {e}"); print(df_plot_final.head())

        else:
             print(f"(!) Advertencia: Faltan columnas base o de coordenadas: {columnas_faltantes_plot}")
             df_plot_final = df_plot_final[[col for col in columnas_plot_final if col in df_plot_final.columns]] # Mantener solo existentes

    else:
        print("(!) Error: Sin coordenadas 2D válidas en 'df_embeddings_combinado'.")
        df_plot_final = pd.DataFrame()
else:
    print("(!) Error: 'df_embeddings_combinado' no encontrado o vacío.")
    df_plot_final = pd.DataFrame()

# --- Fin Chunk 6 (Revisado v3) ---
print("\n--- Fin del Proceso del Chunk 6 Revisado v3 ---")

--- 6.1 Creando DataFrame Final 'df_plot_final' (con Tooltip Dinámico) ---
-> Copiando df_embeddings_combinado (57 filas) a df_plot_final.
   -> Creando columna 'tooltip_details' formateada dinámicamente...
      -> Columna 'tooltip_details' creada exitosamente.
(!) Advertencia: Faltan columnas base o de coordenadas: ['umap_x_3d', 'umap_y_3d', 'umap_z_3d', 'pca_x_3d', 'pca_y_3d', 'pca_z_3d']

--- Fin del Proceso del Chunk 6 Revisado v3 ---


In [ ]:
# @title 7 (Revisado v2 - Mejorado): Visualización Jerárquica (Palabras y Unidades)

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import textwrap
import numpy as np # Para verificar NaNs

print("--- 7.1 Preparando Visualización Jerárquica ---")

# --- Parámetros de Visualización ---

# @markdown ---
# @markdown **Selección de Coordenadas:**
# @markdown Elige qué método de reducción de dimensionalidad usar para el gráfico.
# @markdown *   **UMAP:** Generalmente mejor para visualización. Tiende a preservar mejor la estructura local (grupos de puntos cercanos) y también da una buena idea de la estructura global. Bueno para exploración.
# @markdown *   **PCA:** Más rápido de calcular. Preserva la varianza global (las direcciones de mayor dispersión). Útil si la estructura lineal de los datos es importante o como una alternativa si UMAP da resultados extraños.
coords_a_usar = "UMAP" # @param ["UMAP", "PCA"]

# @markdown ---
# @markdown **Apariencia del Gráfico:**
# @markdown Ajusta la opacidad de las líneas que conectan las palabras (0=invisible, 1=opaco):
opacidad_linea = 0.25 # @param {type:"slider", min:0.0, max:1.0, step:0.05}
# @markdown ¿Mostrar palabras directamente en los puntos? (Puede causar solapamiento)
mostrar_texto_en_puntos = False # @param {type:"boolean"}
# @markdown Tamaño de los puntos de las palabras:
tamano_punto_palabra = 11 # @param {type:"slider", min:4, max:20, step:1}
# @markdown ¿Mostrar marcadores para las Unidades/Texto Completo?
mostrar_marcadores_unidad = True # @param {type:"boolean"}
# @markdown Tamaño de los marcadores de Unidad/Texto Completo:
tamano_punto_unidad = 23 # @param {type:"slider", min:8, max:24, step:1}
# @markdown ---

fig_combinada = None # Inicializar figura

# Verificar DataFrame y columnas necesarias
if 'df_embeddings_combinado' not in globals() or not isinstance(df_embeddings_combinado, pd.DataFrame) or df_embeddings_combinado.empty:
    print("(!) Error: DataFrame 'df_embeddings_combinado' no encontrado o vacío.")
elif 'tipo' not in df_embeddings_combinado.columns or 'embedding_vector' not in df_embeddings_combinado.columns:
     print("(!) Error: Faltan columnas 'tipo' o 'embedding_vector' en 'df_embeddings_combinado'.")
else:
    # Determinar columnas X e Y y verificar su existencia
    x_col = 'umap_x' if coords_a_usar == "UMAP" else 'pca_x'
    y_col = 'umap_y' if coords_a_usar == "UMAP" else 'pca_y'
    coord_label = coords_a_usar

    if x_col not in df_embeddings_combinado.columns or y_col not in df_embeddings_combinado.columns:
        print(f"(!) Error: Columnas de coordenadas '{x_col}' o '{y_col}' no existen.")
    # Verificar que haya al menos algunos valores no nulos en las coordenadas seleccionadas
    elif df_embeddings_combinado[x_col].isnull().all() or df_embeddings_combinado[y_col].isnull().all():
         print(f"(!) Advertencia: Coordenadas {coord_label} seleccionadas contienen solo NaN.")
         # No se puede graficar
    else:
        print(f"-> Usando coordenadas {coord_label}. Opacidad línea: {opacidad_linea}. Mostrar texto: {mostrar_texto_en_puntos}.")

        # Filtrar filas donde las coordenadas seleccionadas son NaN (no se pueden graficar)
        df_plot_valid = df_embeddings_combinado.dropna(subset=[x_col, y_col]).copy()
        if len(df_plot_valid) < len(df_embeddings_combinado):
             print(f"(!) Advertencia: Se omitieron {len(df_embeddings_combinado) - len(df_plot_valid)} elementos con coordenadas NaN.")

        if df_plot_valid.empty:
             print("(!) No quedan elementos válidos para graficar después de quitar NaN.")
        else:
            # --- 7.3 Crear Gráfico Jerárquico ---
            print("-> Creando gráfico jerárquico interactivo...")
            try:
                fig_combinada = go.Figure()
                colores_plotly = px.colors.qualitative.Plotly # Usar una paleta estándar

                # Asegurar orden para las líneas: por índice de unidad, luego por índice de palabra
                df_plot_valid['id_unidad_num'] = df_plot_valid.apply(lambda row: 0 if row['tipo']=='TextoCompleto' else (row['indice_ref'] if row['tipo']=='Unidad' else np.inf), axis=1)
                df_plot_valid['indice_palabra_num'] = df_plot_valid.apply(lambda row: row['indice_ref'] if row['tipo']=='Palabra' else np.inf, axis=1)
                df_plot_valid_sorted = df_plot_valid.sort_values(by=['id_unidad_num', 'indice_palabra_num']).reset_index()

                # --- Capa 1: Líneas conectando palabras dentro de cada unidad ---
                # Necesitamos agrupar por unidad. El texto completo es una unidad (índice 0).
                # Las unidades detectadas tienen índice 1, 2, ...
                # Si solo hay texto completo, su índice_ref es 0. Si hay unidades, van de 1 en adelante.
                # Si hay palabras, su índice_ref es el índice de palabra. Necesitamos mapear palabras a unidades.

                # Mapeo (aproximado): Asignar palabras a la unidad más cercana en el índice original
                # Esta lógica asume que parsed_json (de donde vienen las palabras) y
                # lista_unidades_detectadas (de donde vienen las unidades) están relacionadas con la estructura.
                # Es una simplificación; un mapeo perfecto requeriría info del paso de segmentación.

                # Heurística simple: Dibujar una línea continua entre todas las palabras si no hay unidades claras
                palabras_df = df_plot_valid_sorted[df_plot_valid_sorted['tipo'] == 'Palabra']
                if not palabras_df.empty:
                    fig_combinada.add_trace(go.Scatter(
                        x=palabras_df[x_col],
                        y=palabras_df[y_col],
                        mode='lines',
                        line=dict(color=f'rgba(0,0,0,{opacidad_linea})', width=1.5), # Usar opacidad del @markdown
                        hoverinfo='none',
                        showlegend=False
                    ))
                    print(f"   -> Añadida línea conectando {len(palabras_df)} palabras.")


                # --- Capa 2: Puntos de las Palabras ---
                if not palabras_df.empty:
                    # Usar px.scatter para generar trazas por categoría fácilmente
                    scatter_palabras = px.scatter(
                        palabras_df, x=x_col, y=y_col, color='categoria',
                        hover_name='texto', # <--- CORREGIDO
                        hover_data={'categoria': True, 'indice_ref': True, x_col: ':.3f', y_col: ':.3f'},
                        text='texto' # <--- CORREGIDO (El texto se mostrará o no según el update_traces)
                    )
                    for trace in scatter_palabras.data:
                        trace.update(marker_size=tamano_punto_palabra) # Aplicar tamaño
                        fig_combinada.add_trace(trace)
                    print(f"   -> Añadidos {len(palabras_df)} puntos de palabras coloreados por categoría.")


                # --- Capa 3: Marcadores de Unidades y Texto Completo ---
                if mostrar_marcadores_unidad:
                    unidades_df = df_plot_valid_sorted[df_plot_valid_sorted['tipo'] == 'Unidad']
                    texto_completo_df = df_plot_valid_sorted[df_plot_valid_sorted['tipo'] == 'TextoCompleto']

                    if not unidades_df.empty:
                        fig_combinada.add_trace(go.Scatter(
                            x=unidades_df[x_col], y=unidades_df[y_col],
                            mode='markers',
                            marker=dict(symbol='star', size=tamano_punto_unidad, color='orange',
                                        line=dict(width=1, color='DarkSlateGrey')),
                            name='Unidad Detectada', # Nombre para la leyenda
                            text=[f"Unidad {idx}" for idx in unidades_df['indice_ref']], # Texto para hover
                            hoverinfo='text+x+y'
                        ))
                        print(f"   -> Añadidos {len(unidades_df)} marcadores para Unidades.")

                    if not texto_completo_df.empty:
                        fig_combinada.add_trace(go.Scatter(
                            x=texto_completo_df[x_col], y=texto_completo_df[y_col],
                            mode='markers',
                            marker=dict(symbol='diamond', size=tamano_punto_unidad, color='green',
                                        line=dict(width=1, color='DarkSlateGrey')),
                            name='Texto Completo',
                            text='Texto Completo',
                            hoverinfo='text+x+y'
                        ))
                        print(f"   -> Añadido marcador para Texto Completo.")


                # --- 7.4 Ajustar Diseño y Apariencia ---
                # Ocultar/mostrar texto en puntos de palabras
                fig_combinada.update_traces(
                    text=None if not mostrar_texto_en_puntos else 'default', # 'default' usa el texto asignado
                    textposition='top center',
                    textfont_size=9,
                    selector=dict(type='scatter', mode='markers') # Aplicar solo a trazas con marcadores
                 )

                # Layout General
                titulo_grafico = f"Mapa Semántico Jerárquico ({coord_label})"
                if 'frase_seleccionada' in globals():
                     titulo_grafico += f"<br>Texto: <i>{textwrap.shorten(frase_seleccionada, width=90)}</i>"

                fig_combinada.update_layout(
                    title=titulo_grafico,
                    hovermode='closest',
                    xaxis=dict(showticklabels=False, showgrid=False, zeroline=False, title=None),
                    yaxis=dict(showticklabels=False, showgrid=False, zeroline=False, title=None),
                    legend_title_text='Tipo / Categoría',
                    template='plotly_white',
                    margin=dict(l=20, r=20, t=90, b=20) # Más margen para título
                )

                print("-> Gráfico jerárquico creado/actualizado.")

            except Exception as e:
                print(f"(!) Error inesperado al crear el gráfico Plotly jerárquico: {e}")
                # import traceback
                # traceback.print_exc() # Descomentar para ver el traceback completo si hay errores difíciles
                fig_combinada = None

# --- 7.5 Mostrar Contexto y Gráfico ---
# (El código para mostrar Frase Original y Meta-CoT es el mismo que antes)
# ... (código para imprimir contexto) ...
print("\n" + "="*70)
print("Contexto para la Visualización Jerárquica:")
print("="*70)
if 'frase_seleccionada' in globals(): print(f"Texto Original Seleccionado:\n{textwrap.fill(frase_seleccionada, width=80)}\n")
else: print("Texto Original no disponible.")
print("-"*70)
print("Meta-Análisis CoT (Resumen):")
print("-"*70)
if 'meta_cot_resumen' in globals() and meta_cot_resumen:
    from IPython.display import display, Markdown; display(Markdown(meta_cot_resumen))
else: print("(Meta-Análisis CoT no disponible)")
print("="*70 + "\n")

# Mostrar el gráfico
if fig_combinada is not None:
    print("\n--- Mostrando Gráfico Interactivo Jerárquico ---")
    print("Puntos: Palabras (color=categoría), Estrellas: Unidades, Diamante: Texto Completo.")
    print("Líneas conectan palabras en secuencia.")
    print("(Pasa el ratón sobre los puntos para ver detalles)")
    fig_combinada.show()
else:
    print("\n(!) El gráfico jerárquico no pudo ser generado o mostrado.")


# --- Fin Chunk 7 (Revisado v2 - Mejorado) ---
print("\n--- Fin del Proceso del Chunk 7 Revisado v2 y Mejorado ---")

In [106]:
# @title 9. Reducción de Dimensionalidad a 3D (UMAP y PCA)

import numpy as np
import pandas as pd
import umap
from sklearn.decomposition import PCA

print("--- 9.1 Preparando embeddings COMBINADOS para reducción a 3D ---")

embeddings_3d_umap = None
embeddings_3d_pca = None
reduction_3d_successful = False

# Verificar DataFrame combinado
if 'df_embeddings_combinado' in globals() and isinstance(df_embeddings_combinado, pd.DataFrame) and not df_embeddings_combinado.empty \
   and 'embedding_vector' in df_embeddings_combinado.columns:

    valid_embeddings_series_3d = df_embeddings_combinado['embedding_vector'].dropna()
    if not valid_embeddings_series_3d.empty:
        embedding_vectors_list_3d = valid_embeddings_series_3d.tolist()
        try:
            valid_vectors_3d = [vec for vec in embedding_vectors_list_3d if isinstance(vec, np.ndarray)]
            if valid_vectors_3d:
                embeddings_array_3d = np.stack(valid_vectors_3d, axis=0)
                print(f"-> Array de embeddings preparado para 3D. Forma: {embeddings_array_3d.shape}")

                # Necesitamos al menos 3 puntos para PCA 3D, y preferiblemente > n_neighbors para UMAP 3D
                if embeddings_array_3d.shape[0] >= 3:
                    reduction_3d_possible = True
                else:
                    print("(!) No hay suficientes puntos (se necesitan >= 3) para reducción a 3D significativa.")
                    reduction_3d_possible = False

            else: reduction_3d_possible = False
        except Exception as e:
            print(f"(!) Error al preparar array para 3D: {e}"); reduction_3d_possible = False
    else:
        print("(!) No hay embeddings válidos en 'df_embeddings_combinado'."); reduction_3d_possible = False
else:
    print("(!) DataFrame 'df_embeddings_combinado' no encontrado o inválido."); reduction_3d_possible = False

# --- 9.2 Aplicar Reducción a 3D (si es posible) ---
if reduction_3d_possible:
    print("\n--- 9.2 Aplicando Reducción de Dimensionalidad a 3D ---")

    # --- UMAP 3D ---
    print("   -> Calculando reducción 3D con UMAP...")
    try:
        n_samples = embeddings_array_3d.shape[0]
        # UMAP 3D puede necesitar más vecinos, pero mantenemos la heurística
        n_neighbors_umap_3d = min(15, max(2, n_samples - 1)) if n_samples > 2 else 1
        print(f"      (Usando n_neighbors={n_neighbors_umap_3d})")

        umap_reducer_3d = umap.UMAP(
            n_components=3, # <--- CAMBIO CLAVE
            n_neighbors=n_neighbors_umap_3d, min_dist=0.1,
            metric='cosine', random_state=42, n_jobs=1
        )
        embeddings_3d_umap = umap_reducer_3d.fit_transform(embeddings_array_3d)
        print(f"      -> Reducción UMAP 3D completada. Forma: {embeddings_3d_umap.shape}")
        reduction_3d_successful = True
    except Exception as e:
        print(f"   (!) Error durante la reducción UMAP 3D: {e}")
        embeddings_3d_umap = None
        if not reduction_3d_successful: print("      Intentando PCA 3D como alternativa...")

    # --- PCA 3D ---
    print("\n   -> Calculando reducción 3D con PCA...")
    try:
        pca_reducer_3d = PCA(
            n_components=3, # <--- CAMBIO CLAVE
            random_state=42
        )
        embeddings_3d_pca = pca_reducer_3d.fit_transform(embeddings_array_3d)
        print(f"      -> Reducción PCA 3D completada. Forma: {embeddings_3d_pca.shape}")
        if not reduction_3d_successful: reduction_3d_successful = True
    except Exception as e:
        print(f"   (!) Error durante la reducción PCA 3D: {e}")
        embeddings_3d_pca = None

# --- 9.3 Añadir Coordenadas 3D al DataFrame Final ---
# Usaremos el df_plot_final del Chunk 6 (que ya tiene otras infos)
if 'df_plot_final' in globals() and isinstance(df_plot_final, pd.DataFrame) and not df_plot_final.empty:
    if reduction_3d_successful:
        print("\n--- 9.3 Añadiendo coordenadas 3D al DataFrame 'df_plot_final' ---")
        # Asegurarse de alinear con los índices correctos (los que tenían embeddings válidos)
        indices_validos_3d = valid_embeddings_series_3d.index

        try:
            # Añadir UMAP 3D
            if embeddings_3d_umap is not None and embeddings_3d_umap.shape[0] == len(indices_validos_3d):
                 df_coords_umap_3d = pd.DataFrame(embeddings_3d_umap, columns=['umap_x_3d', 'umap_y_3d', 'umap_z_3d'], index=indices_validos_3d)
                 df_plot_final = df_plot_final.join(df_coords_umap_3d)
                 print("   -> Coordenadas UMAP 3D añadidas.")
            else:
                 print("   (!) No se añadieron coordenadas UMAP 3D.")
                 if 'umap_x_3d' not in df_plot_final.columns: df_plot_final['umap_x_3d'] = np.nan # Añadir NaN si no existen
                 if 'umap_y_3d' not in df_plot_final.columns: df_plot_final['umap_y_3d'] = np.nan
                 if 'umap_z_3d' not in df_plot_final.columns: df_plot_final['umap_z_3d'] = np.nan

            # Añadir PCA 3D
            if embeddings_3d_pca is not None and embeddings_3d_pca.shape[0] == len(indices_validos_3d):
                 df_coords_pca_3d = pd.DataFrame(embeddings_3d_pca, columns=['pca_x_3d', 'pca_y_3d', 'pca_z_3d'], index=indices_validos_3d)
                 df_plot_final = df_plot_final.join(df_coords_pca_3d)
                 print("   -> Coordenadas PCA 3D añadidas.")
            else:
                 print("   (!) No se añadieron coordenadas PCA 3D.")
                 if 'pca_x_3d' not in df_plot_final.columns: df_plot_final['pca_x_3d'] = np.nan
                 if 'pca_y_3d' not in df_plot_final.columns: df_plot_final['pca_y_3d'] = np.nan
                 if 'pca_z_3d' not in df_plot_final.columns: df_plot_final['pca_z_3d'] = np.nan

        except Exception as e:
              print(f"(!) Error al añadir coordenadas 3D al DataFrame: {e}")

    else: # Si la reducción 3D no fue posible o falló
        print("\n--- 9.3 No se realizó o falló la reducción a 3D ---")
        # Añadir columnas NaN si no existen
        for col in ['umap_x_3d', 'umap_y_3d', 'umap_z_3d', 'pca_x_3d', 'pca_y_3d', 'pca_z_3d']:
             if col not in df_plot_final.columns: df_plot_final[col] = np.nan
        print("   -> Columnas de coordenadas 3D añadidas como NaN.")
else:
     print("(!) Error: No se encontró 'df_plot_final' para añadir coordenadas 3D.")


# --- Fin Chunk 9 ---
print("\n--- Fin del Proceso del Chunk 9 (Reducción 3D) ---")

--- 9.1 Preparando embeddings COMBINADOS para reducción a 3D ---
-> Array de embeddings preparado para 3D. Forma: (57, 768)

--- 9.2 Aplicando Reducción de Dimensionalidad a 3D ---
   -> Calculando reducción 3D con UMAP...
      (Usando n_neighbors=15)
      -> Reducción UMAP 3D completada. Forma: (57, 3)

   -> Calculando reducción 3D con PCA...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



      -> Reducción PCA 3D completada. Forma: (57, 3)

--- 9.3 Añadiendo coordenadas 3D al DataFrame 'df_plot_final' ---
   -> Coordenadas UMAP 3D añadidas.
   -> Coordenadas PCA 3D añadidas.

--- Fin del Proceso del Chunk 9 (Reducción 3D) ---


In [107]:
# @title 10 (v4 - Final): Visualización 3D con Tooltips Dinámicos

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import textwrap
import numpy as np
from IPython.display import display, Markdown # Para renderizar Markdown si muestras contexto aquí

print("--- 10.1 Preparando Visualización 3D (v4 - Tooltips Dinámicos) ---")

# --- Parámetros de Visualización 3D ---
# @markdown (Mismos parámetros @markdown que antes)
# @markdown ---
coords_3d_a_usar = "UMAP" # @param ["UMAP", "PCA"]
# @markdown ---
opacidad_linea_3d = 0.3 # @param {type:"slider", min:0.0, max:1.0, step:0.05}
# ... (resto de parámetros) ...
mostrar_texto_en_puntos = False # @param {type:"boolean"}
tamano_punto_palabra_3d = 7 # @param {type:"slider", min:2, max:15, step:1}
mostrar_marcadores_unidad_3d = True # @param {type:"boolean"}
tamano_punto_unidad_3d = 10 # @param {type:"slider", min:4, max:20, step:1}
# @markdown ---

fig_3d = None

# Verificar DataFrame y columnas
# Verificar que df_plot_final y la columna tooltip_details existan
if 'df_plot_final' not in globals() or not isinstance(df_plot_final, pd.DataFrame) or df_plot_final.empty:
    print("(!) Error: DataFrame 'df_plot_final' no encontrado o vacío. Ejecuta Chunks 6 y 9.")
elif 'tooltip_details' not in df_plot_final.columns:
     print("(!) Error: Falta la columna 'tooltip_details'. Ejecuta el Chunk 6 (Revisado v3) correctamente.")
# ... (resto de verificaciones iniciales de coordenadas 3D) ...
else:
    x_col_3d = f'{coords_3d_a_usar.lower()}_x_3d'
    y_col_3d = f'{coords_3d_a_usar.lower()}_y_3d'
    z_col_3d = f'{coords_3d_a_usar.lower()}_z_3d'
    coord_label_3d = coords_3d_a_usar

    # Verificar existencia y validez de coordenadas 3D
    if not all(col in df_plot_final.columns for col in [x_col_3d, y_col_3d, z_col_3d]):
        print(f"(!) Error: Columnas 3D '{x_col_3d}', '{y_col_3d}', '{z_col_3d}' no existen. Ejecuta Chunk 9.")
    elif df_plot_final[[x_col_3d, y_col_3d, z_col_3d]].isnull().all().any():
        print(f"(!) Advertencia: Coordenadas 3D {coord_label_3d} con NaN.")
    else:
        print(f"-> Usando coordenadas 3D {coord_label_3d}.")
        # Usar .copy() para evitar warnings
        df_plot_3d_valid = df_plot_final.dropna(subset=[x_col_3d, y_col_3d, z_col_3d]).copy()
        # ... (código de advertencia si se omitieron NaNs) ...

        if df_plot_3d_valid.empty:
             print("(!) No quedan elementos válidos para graficar en 3D.")
        else:
            print("-> Creando gráfico 3D interactivo (v4)...")
            try:
                fig_3d = go.Figure()
                # Reutilizar o regenerar mapa de colores
                if 'color_map' not in globals() or not color_map:
                    temp_palabras_df = df_plot_3d_valid[df_plot_3d_valid['tipo'] == 'Palabra']
                    if not temp_palabras_df.empty:
                        ordered_categories_3d = sorted(temp_palabras_df['categoria'].unique())
                        color_sequence_3d = px.colors.qualitative.Plotly
                        color_map = {cat: color_sequence_3d[i % len(color_sequence_3d)] for i, cat in enumerate(ordered_categories_3d)}
                        print("   -> Mapa de colores regenerado para 3D.")
                    else: color_map = {}
                else:
                    color_map_3d = color_map
                    print("   -> Usando mapa de colores existente.")

                # Ordenar para líneas
                df_plot_3d_valid_sorted = df_plot_3d_valid.sort_values(by='indice_ref').reset_index(drop=True)
                palabras_df_3d = df_plot_3d_valid_sorted[df_plot_3d_valid_sorted['tipo'] == 'Palabra']

                # --- Capa 1: Líneas 3D conectando palabras ---
                # ... (código sin cambios) ...
                if not palabras_df_3d.empty:
                    fig_3d.add_trace(go.Scatter3d(x=palabras_df_3d[x_col_3d], y=palabras_df_3d[y_col_3d], z=palabras_df_3d[z_col_3d],
                                                  mode='lines', line=dict(color=f'rgba(0,0,0,{opacidad_linea_3d})', width=2),
                                                  hoverinfo='none', showlegend=False))
                    print(f"   -> Añadida línea 3D conectando {len(palabras_df_3d)} palabras.")


                # --- Capa 2: Puntos 3D de las Palabras (Usando Tooltip Dinámico) --- ## ACTUALIZADO ##
                if not palabras_df_3d.empty and color_map_3d:
                    print(f"   -> Añadiendo puntos 3D para palabras...")
                    for categoria, color in color_map_3d.items():
                        df_categoria_3d = palabras_df_3d[palabras_df_3d['categoria'] == categoria]
                        if not df_categoria_3d.empty:
                             # Asegurarse que las columnas para customdata existen
                             cols_custom_data = ['texto', 'tooltip_details', 'indice_ref']
                             if all(c in df_categoria_3d.columns for c in cols_custom_data):
                                 fig_3d.add_trace(go.Scatter3d(
                                     x=df_categoria_3d[x_col_3d], y=df_categoria_3d[y_col_3d], z=df_categoria_3d[z_col_3d],
                                     mode='markers',
                                     marker=dict(color=color, size=tamano_punto_palabra_3d, opacity=0.9,
                                                 line=dict(width=0.5, color='DarkSlateGrey')),
                                     name=categoria,
                                     customdata=df_categoria_3d[cols_custom_data], # Pasar las 3 columnas
                                     hovertemplate=( # Usar el hovertemplate corregido
                                         "<b><u>%{customdata[0]}</u></b><br>" # Palabra subrayada/negrita (índice 0)
                                         f"<b>Categoría:</b> {categoria} | <b>Índice:</b> %{{customdata[2]}}<br>" # Cat (fija) e Índice (índice 2)
                                         "<br>" # Separador
                                         "%{customdata[1]}" # Detalles formateados (índice 1)
                                         "<br><br>" # Separador antes de coords
                                         f"X: %{{x:.3f}}<br>" # Coordenadas
                                         f"Y: %{{y:.3f}}<br>"
                                         f"Z: %{{z:.3f}}"
                                         "<extra></extra>"
                                     )
                                 ))
                             else:
                                 print(f"(!) Advertencia: Faltan columnas en customdata para categoría '{categoria}'. Se omite tooltip detallado.")
                                 # Añadir traza con tooltip básico si faltan datos
                                 fig_3d.add_trace(go.Scatter3d(x=df_categoria_3d[x_col_3d], y=df_categoria_3d[y_col_3d], z=df_categoria_3d[z_col_3d],
                                                              mode='markers', marker=dict(color=color, size=tamano_punto_palabra_3d), name=categoria,
                                                              hovertext=df_categoria_3d['texto'])) # Tooltip básico con el texto

                    print(f"      -> Puntos 3D añadidos para {len(color_map_3d)} categorías con tooltips dinámicos.")
                # ... (código fallback sin cambios) ...

                # --- Capa 3: Marcadores 3D Unidades / Texto Completo (Usando Tooltip Dinámico) --- ## ACTUALIZADO ##
                if mostrar_marcadores_unidad_3d:
                    unidades_df_3d = df_plot_3d_valid_sorted[df_plot_3d_valid_sorted['tipo'] == 'Unidad'].copy() # Usar copia
                    texto_completo_df_3d = df_plot_3d_valid_sorted[df_plot_3d_valid_sorted['tipo'] == 'TextoCompleto'].copy() # Usar copia
                    primera_palabra_3d = palabras_df_3d.iloc[0] if not palabras_df_3d.empty else None

                    # Marcador Texto Completo
                    if not texto_completo_df_3d.empty:
                        tc_row_3d = texto_completo_df_3d.iloc[0]
                        # Usar tooltip_details directamente si existe
                        tooltip_tc = tc_row_3d.get('tooltip_details', "Info no disponible")
                        fig_3d.add_trace(go.Scatter3d(
                            x=[tc_row_3d[x_col_3d]], y=[tc_row_3d[y_col_3d]], z=[tc_row_3d[z_col_3d]], mode='markers',
                            marker=dict(symbol='diamond', size=tamano_punto_unidad_3d, color='green', line_width=1, line_color='DarkSlateGrey'),
                            name='Texto Completo',
                            customdata=[[tooltip_tc]], # Pasar detalles formateados
                            hovertemplate="%{customdata[0]}<extra></extra>" # Mostrar detalles directamente
                        ))
                        print(f"   -> Añadido marcador 3D Texto Completo (diamond).")
                        # ... (código línea punteada sin cambios) ...
                        if primera_palabra_3d is not None:
                             fig_3d.add_trace(go.Scatter3d(x=[tc_row_3d[x_col_3d], primera_palabra_3d[x_col_3d]], y=[tc_row_3d[y_col_3d], primera_palabra_3d[y_col_3d]], z=[tc_row_3d[z_col_3d], primera_palabra_3d[z_col_3d]], mode='lines', line=dict(color='rgba(0,128,0,0.5)', width=1.5, dash='dot'), hoverinfo='none', showlegend=False)); print(f"      -> Añadida línea 3D conexión a 1ra palabra.")


                    # Marcadores Unidades Detectadas
                    if not unidades_df_3d.empty:
                         # Asegurar que tooltip_details existe
                         if 'tooltip_details' not in unidades_df_3d.columns: unidades_df_3d['tooltip_details'] = "Info no disponible"
                         # Añadir indice_ref a customdata si no está ya implícito
                         if 'indice_ref' not in unidades_df_3d.columns: unidades_df_3d['indice_ref'] = 'N/A' # Fallback

                         fig_3d.add_trace(go.Scatter3d(
                             x=unidades_df_3d[x_col_3d], y=unidades_df_3d[y_col_3d], z=unidades_df_3d[z_col_3d], mode='markers',
                             marker=dict(symbol='cross', size=tamano_punto_unidad_3d, color='orange', line_width=1),
                             name='Unidad Detectada',
                             # Pasar tooltip_details y el indice_ref
                             customdata=unidades_df_3d[['tooltip_details', 'indice_ref']],
                             hovertemplate=(
                                 "<b>Unidad Detectada #%{customdata[1]}</b><br>" # Índice/número de unidad
                                 "<br>" # Separador
                                 "%{customdata[0]}" # Mostrar detalles base formateados
                                 "<extra></extra>"
                             )
                         ))
                         print(f"   -> Añadidos {len(unidades_df_3d)} marcadores 3D Unidades (cross).")


                # --- 10.4 Ajustar Layout 3D y Texto ---
                # ... (Código sin cambios significativos, asegurar selectores correctos) ...
                fig_3d.update_traces(text=df_plot_3d_valid_sorted['texto'] if mostrar_texto_en_puntos else None, textposition='top center', textfont_size=9, selector=dict(type='scatter3d', mode='markers'))
                if not mostrar_texto_en_puntos: fig_3d.update_traces(text=None, selector=dict(type='scatter3d', mode='markers'))
                # ... (resto del layout) ...
                titulo_grafico_3d = f"Mapa Semántico 3D ({coord_label_3d})" #...
                if 'frase_seleccionada' in globals(): titulo_grafico_3d += f"<br>Texto: <i>{textwrap.shorten(frase_seleccionada, width=120)}</i>" #...
                fig_3d.update_layout(title=titulo_grafico_3d, margin=dict(l=10, r=10, t=90, b=10), scene=dict(xaxis_title=f'{coord_label_3d} X', yaxis_title=f'{coord_label_3d} Y', zaxis_title=f'{coord_label_3d} Z', xaxis=dict(showticklabels=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="lightgrey"), yaxis=dict(showticklabels=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="lightgrey"), zaxis=dict(showticklabels=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="lightgrey"), aspectratio=dict(x=1, y=1, z=0.7)), legend_title_text='Tipo / Categoría', template='plotly_white')


                print("-> Gráfico 3D (v4) creado/actualizado.")

            except Exception as e:
                print(f"(!) Error inesperado al crear el gráfico Plotly 3D v4: {e}")
                import traceback
                traceback.print_exc()
                fig_3d = None

# --- 10.5 Mostrar Contexto y Gráfico 3D ---
# ... (Código para mostrar contexto sin cambios) ...
print("\n" + "="*70); print("Contexto para la Visualización 3D:"); print("="*70) #... (resto de impresión de contexto) ...
if 'frase_seleccionada' in globals(): print(f"Texto Original Seleccionado:\n{textwrap.fill(frase_seleccionada, width=80)}\n") #...
else: print("Texto Original no disponible.") #...
print("-"*70); print("Meta-Análisis CoT (Resumen):"); print("-"*70) #...
if 'meta_cot_resumen' in globals() and meta_cot_resumen: display(Markdown(meta_cot_resumen)) #...
else: print("(Meta-Análisis CoT no disponible)") #...
print("="*70 + "\n") #...

# Mostrar el gráfico 3D
if fig_3d is not None:
    print("\n--- Mostrando Gráfico Interactivo 3D (v4 - Tooltips Dinámicos) ---")
    fig_3d.show()
else:
    print("\n(!) El gráfico 3D v4 no pudo ser generado o mostrado.")

# --- Fin Chunk 10 (v4) ---
print("\n--- Fin del Proceso del Chunk 10 (v4 - Tooltips Dinámicos) ---")

--- 10.1 Preparando Visualización 3D (v4 - Tooltips Dinámicos) ---
-> Usando coordenadas 3D UMAP.
-> Creando gráfico 3D interactivo (v4)...
   -> Usando mapa de colores existente.
   -> Añadida línea 3D conectando 54 palabras.
   -> Añadiendo puntos 3D para palabras...
      -> Puntos 3D añadidos para 7 categorías con tooltips dinámicos.
   -> Añadido marcador 3D Texto Completo (diamond).
      -> Añadida línea 3D conexión a 1ra palabra.
   -> Añadidos 2 marcadores 3D Unidades (cross).
-> Gráfico 3D (v4) creado/actualizado.

Contexto para la Visualización 3D:
Texto Original Seleccionado:
Para realizar la configuración óptima del dispositivo, asegúrese de verificar
que los puertos de entrada, los cuales suelen ubicarse en la parte posterior
según el modelo adquirido, estén correctamente alineados con los conectores del
cable HDMI, evitando así posibles fallos de transmisión de datos durante el
proceso de sincronización.

------------------------------------------------------------------

* Naturaleza: Instrucción técnica formal para la configuración de un dispositivo electrónico.  El texto pertenece a un manual de usuario o guía de configuración, con el objetivo de guiar al usuario en la conexión correcta del dispositivo.
* Ambigüedad: Ausencia de ambigüedad significativa. El significado es claro y conciso, dirigido a evitar problemas de conexión.
* Complejidad:  Estructura gramatical compleja, con una oración principal y una subordinada (que a su vez contiene una oración relativa).  Sin embargo, la complejidad gramatical no afecta la claridad del mensaje.  El texto utiliza vocabulario técnico específico del ámbito de la electrónica.
* Conclusión Semántica: El texto cumple eficazmente su función instructiva, proporcionando instrucciones precisas y concisas para evitar fallos en la conexión del dispositivo.


* Palabras unidas en sentido figurado: No hay palabras o frases unidas en sentido figurado. El lenguaje es completamente literal y técnico.  No presenta características literarias, poéticas o musicales.



--- Mostrando Gráfico Interactivo 3D (v4 - Tooltips Dinámicos) ---



--- Fin del Proceso del Chunk 10 (v4 - Tooltips Dinámicos) ---
